# Convertir datos y graficar

In [21]:
import mysql.connector
import pandas as pd
from datetime import datetime, timedelta

db = mysql.connector.connect(host="192.168.0.12",    #  host
                     user="operario",         #  usuario
                     passwd="",  #  password
                     db="dbp8100")        #  nombre de la base de datos

cur = db.cursor()

In [22]:
fecha_actual = datetime.now()
fecha_ayer = fecha_actual - timedelta(days=1)
fecha_actual = fecha_actual.strftime("%Y-%m-%d")
fecha_ayer = fecha_ayer.strftime("%Y-%m-%d")

# fecha_actual = '2025-02-07'

print(fecha_actual)
print(fecha_ayer)

2025-03-14
2025-03-13


In [23]:
# Extraer datos y convertir a df

# Datos mysql
cur.execute('''
            SELECT                 
                tareaseje.NroID,
                tareaseje.Fecha AS Fecha,
                tareaseje.Hora AS Hora,
                formulas.NroID AS IDF,
                formulas.Nombre AS Nombre,
                tareaseje.Set AS Programado,
                (SELECT SUM(dcaptura.Valor) FROM dbp8100.dcaptura WHERE dcaptura.IDT = tareaseje.NroID) AS Dosificado,
                tareaseje.Tiempo AS Tiempo
            FROM 
                dbp8100.tareaseje AS tareaseje
            JOIN 
                dbp8100.formulas AS formulas ON formulas.NroID = tareaseje.IDF
            WHERE
                (tareaseje.Fecha = %s AND tareaseje.Hora >= '23:00:00') OR 
                (tareaseje.Fecha > %s AND tareaseje.Fecha <= %s)
            GROUP BY
                tareaseje.NroID
            ''',(fecha_ayer, fecha_ayer, fecha_actual),)
resultados = cur.fetchall()
# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)

# Cerrar el cursor y la conexión
cur.close()
db.close()

# Mostrar el DataFrame
print(df)

   NroID       Fecha            Hora   IDF  \
0  25366  2025-03-13 0 days 23:07:16  1208   
1  25367  2025-03-13 0 days 23:51:11  1079   
2  25368  2025-03-13 0 days 23:55:57  1079   
3  25369  2025-03-14 0 days 06:30:55  1079   
4  25370  2025-03-14 0 days 07:41:14  1244   
5  25371  2025-03-14 0 days 08:24:42  1244   
6  25372  2025-03-14 0 days 09:05:00    86   
7  25373  2025-03-14 0 days 09:55:25    86   

                                Nombre  Programado  Dosificado          Tiempo  
0          TERMINACIÓN (LA MALVINA)...      7000.0     6998.30 0 days 00:40:27  
1             CREC. MELAZA 12 % A.A...     29000.0         NaN 0 days 00:00:04  
2             CREC. MELAZA 12 % A.A...     29000.0    29353.20 0 days 06:34:50  
3             CREC. MELAZA 12 % A.A...      3000.0     3013.05 0 days 01:02:47  
4  RECRIA 14 SIN PELETEAR (quebradora)      5000.0     3009.40 0 days 00:42:33  
5  RECRIA 14 SIN PELETEAR (quebradora)      2000.0     2009.60 0 days 00:29:10  
6      DESTETE PRE

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   NroID       8 non-null      int64          
 1   Fecha       8 non-null      object         
 2   Hora        8 non-null      timedelta64[ns]
 3   IDF         8 non-null      int64          
 4   Nombre      8 non-null      object         
 5   Programado  8 non-null      float64        
 6   Dosificado  8 non-null      float64        
 7   Tiempo      8 non-null      timedelta64[ns]
dtypes: float64(2), int64(2), object(2), timedelta64[ns](2)
memory usage: 644.0+ bytes


In [5]:
ultima_hora = df.loc[df.index[-1], 'Tiempo']
print(ultima_hora)

0 days 00:33:24


In [6]:
pd.isna(ultima_hora)

False

In [22]:
hora_actual = pd.to_timedelta(datetime.now().strftime('%H:%M:%S'))
print(hora_actual)

0 days 07:13:52


In [23]:
df

,NroID,Fecha,Hora,IDF,Nombre,Programado,Dosificado,Tiempo
0,25183,2025-02-11,0 days 03:03:57,1244,RECRIA 14 SIN PELETEAR (quebradora),14000.0,14019.60,0 days 02:03:17
1,25184,2025-02-11,0 days 05:16:43,52,EQUINO SERVICIO,10000.0,10069.75,0 days 01:11:41
2,25185,2025-02-11,0 days 06:34:07,1120,RECRIA 16 AL 15% A.A....,22000.0,3012.10,0 days 00:00:00


In [32]:
valor = df.loc[df.index[-1], 'Tiempo']
print(valor)
if valor == pd.Timedelta(seconds=0):
    print('Es cero')
else:
    print('Que paso')

0 days 00:00:00
Es cero


In [10]:
#Corregir los tiempos 0
# Identificar filas con tiempo 0
mask = df['Tiempo'] == pd.Timedelta(0)

# Calcular la diferencia de hora con la siguiente fila
df.loc[mask, 'Tiempo'] = df['Hora'].shift(-1) - df.loc[mask, 'Hora']

# Para la última fila, calcular diferencia con la hora actual
df.loc[df.index[-1], 'Tiempo'] = hora_actual - df.loc[df.index[-1], 'Hora']

df

,NroID,Fecha,Hora,IDF,Nombre,Programado,Dosificado,Tiempo
0,25183,2025-02-11,0 days 03:03:57,1244,RECRIA 14 SIN PELETEAR (quebradora),14000.0,14019.60,0 days 02:03:17
1,25184,2025-02-11,0 days 05:16:43,52,EQUINO SERVICIO,10000.0,10069.75,0 days 01:11:41
2,25185,2025-02-11,0 days 06:34:07,1120,RECRIA 16 AL 15% A.A....,22000.0,3012.10,0 days 00:39:20


In [11]:
df['Rendimiento'] = (df['Dosificado'] / (df['Tiempo'].astype('int64')/3.6e12))/1000
# df.dropna(inplace=True)
df

,NroID,Fecha,Hora,IDF,Nombre,Programado,Dosificado,Tiempo,Rendimiento
0,25183,2025-02-11,0 days 03:03:57,1244,RECRIA 14 SIN PELETEAR (quebradora),14000.0,14019.60,0 days 02:03:17,6.823112
1,25184,2025-02-11,0 days 05:16:43,52,EQUINO SERVICIO,10000.0,10069.75,0 days 01:11:41,8.428528
2,25185,2025-02-11,0 days 06:34:07,1120,RECRIA 16 AL 15% A.A....,22000.0,3012.10,0 days 00:39:20,4.594729


## Gráficos

In [12]:
import plotly.express as px


In [13]:
nombre = (df.index + 1).astype(str) + " - " + df['Nombre']

# Crear gráfico de barras
fig = px.bar(df, x=nombre, y='Dosificado', title='Producción por Producto')

# Mostrar el gráfico
fig.show()

In [14]:
hora = df['Hora'].astype('int64')/3.6e12
# Crear gráfico de barras
fig = px.line(df, x=hora, y='Rendimiento', title='Rendimiento por Producto')

# Mostrar el gráfico
fig.show()

### Etiquetas superiores

In [15]:
#Toneladas elaboradas
tn_total = df['Dosificado'].sum()

print(tn_total)

27101.449999999997


In [16]:
# Horas totales
hs_total = df['Tiempo'].astype('int64').sum() / 3.6e12

print(hs_total)

3.905


In [17]:
#Rendimiento promedio
rendimiento_gral = (tn_total / hs_total)/1000
print(rendimiento_gral)

6.940192061459666


In [18]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    value = 200,
    delta = {'reference': 160},
    gauge = {
        'axis': {'visible': False}},
    domain = {'row': 0, 'column': 0}))

fig.add_trace(go.Indicator(
    value = 120,
    gauge = {
        'shape': "bullet",
        'axis' : {'visible': False}},
    domain = {'x': [0.05, 0.5], 'y': [0.15, 0.35]}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = 300,
    domain = {'row': 0, 'column': 1}))

fig.add_trace(go.Indicator(
    mode = "delta",
    value = 40,
    domain = {'row': 1, 'column': 1}))

fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Speed"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }})